### Pandas Lab -- Cleaning, Merging, & Grouping

This lab is designed to introduce students to common use cases for Pandas when working with data:

 - Creating new information out of your existing data set
 - Merging, concatenating, and joining different data sources
 - Grouping -- With both time & non-time based data

### Section I: Creating Data Out of Your Existing Columns

Go ahead and create the following columns in your dataset.

**Column 1:**

  - **Column Name:** Profitable
  - **Values:** `True` if `Profit` > 0, `False` if not.

In [344]:
import pandas as pd
import numpy as np
df = pd.read_excel('../../data/superstore.xls', index_col='Row ID')

In [345]:
# your answer here
df['Profitable'] = np.where(df['Profit'] > 0, True, False)

**Column 2:**

 - **Column Name:** Expected Ship Time
 - **Values:**
   - `0` if `Ship Mode` == `Same Day`
   - `2` if `Ship Mode` == `First Class`
   - `3` if `Ship Mode` == `Second Class`
   - `6` if `Ship Mode` == `Standard Class`
   - `-1` if none of the above.

In [346]:
# your answer here
conditions = [
    df['Ship Mode'] == 'Same Day',
    df['Ship Mode'] == 'First Class',
    df['Ship Mode'] == 'Second Class',
    df['Ship Mode'] == 'Standard Class'
]

results = [0, 2, 3, 6]

df['Expected Ship Time'] = np.select(conditions, results, -1)

**Column 3:**

 - **Column Name:** Actual Ship Time
 - **Values:**
   - `Ship Date` - `Order Date`
 - **Note:** When you subtract these columns, your column will be a **time delta**.  See if you can use the `dt` attribute to convert these values into an integer.  Ie, if your value reads `3 days`, you want that to be 3 instead.  You can read more about different time periods in pandas here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [347]:
# your answer here
df['Actual Ship Time'] = (df['Ship Date'] - df['Order Date']).dt.days

In [348]:
#PANDAS + DATES 
df['Order Date'].dt.month #would grab the month
df['Order Date'].dt.day #would grab the day! 
df['Order Date'].dt.year #would grab the year! 

Row ID
1       2016
2       2016
3       2016
4       2015
5       2015
        ... 
9990    2014
9991    2017
9992    2017
9993    2017
9994    2017
Name: Order Date, Length: 9994, dtype: int64

**Column 4:**

 - **Column Name:** Late
 - **Values:** `True` if `Actual Ship Time` > `Expected Ship Time`, `False` otherwise

In [349]:
# your answer here
df['Late'] = df['Actual Ship Time'] > df['Expected Ship Time']

In [350]:
df['Late'].value_counts()

False    7925
True     2069
Name: Late, dtype: int64

### Section II: Merging Dataframes

This excel spreadsheet has 3 separate sheets.  Look up the documentation on the `pd.read_excel` method on how to load in the other two.  

After that, merge the other two dataframes into your original one, and make sure your original dataset now has the following columns:

 - **Salesperson:** This is the Salesperson in charge of each region.
 - **Returned:** This details whether or not the order was returned.  Fill in null values with the value `no`.
 
Use the `drop()` method if you need to get rid of redundant columns.

**Important:** We want to keep all of the rows in the dataset we first loaded in.  After each merge, it's a good idea to make sure your dataset hasn't shrunk, which will happen if you don't choose the right merge type.  Make sure you have 9,994 rows when you're finished!

In [351]:
df2 = pd.read_excel('../../data/superstore.xls', sheet_name=1)
df3 = pd.read_excel('../../data/superstore.xls', sheet_name=2)
#we're going to do a LEFT join, to keep all the values in the original dataframe
df = df.merge(df2, how='left')
df = df.merge(df3, how='left')

In [352]:
# notice we have lots of empty values when we do the merge
df['Returned'].fillna('No', inplace=True)

df.columns

Index(['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code',
       'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Profitable',
       'Expected Ship Time', 'Actual Ship Time', 'Late', 'Returned', 'Person'],
      dtype='object')

### Section III: Grouping

Use the `groupby` or `resample` method to answer the following questions.

In [353]:
# df.groupby('Region') will return u what u do not wnat 
df.groupby('Region').sum()
#this will group by region and sum up everything within 

,Postal Code,Sales,Quantity,Discount,Profit,Profitable,Expected Ship Time,Actual Ship Time,Late
Region,,,,,,,,,
Central,151786150,501239.8908,8780,558.34,39706.3625,1571.0,10627,9427,503.0
East,50171698,678781.2400,10618,414.00,91522.7800,2276.0,12608,11132,563.0
South,55875052,391721.9050,6209,238.55,46749.4303,1348.0,7299,6412,318.0
West,293739752,725457.8245,12266,350.20,108418.4489,2863.0,14185,12587,685.0


In [354]:
df.groupby(['Region','Category']).sum()

Postal Code        Sales  Quantity  Discount  \
Region  Category                                                        
Central Furniture           31498984  163797.1638      1827    143.04   
        Office Supplies     92822095  167026.4150      5409    359.40   
        Technology          27465071  170416.3120      1544     55.90   
East    Furniture           10407666  208291.2040      2214     92.60   
        Office Supplies     29791451  205516.0550      6462    244.70   
        Technology           9972581  264973.9810      1942     76.70   
South   Furniture           11471303  117298.6840      1291     40.35   
        Office Supplies     34072173  125651.3130      3800    166.60   
        Technology          10331576  148771.9080      1118     31.60   
West    Furniture           64818073  252612.7435      2696     92.90   
        Office Supplies    174087153  220853.2490      7235    177.10   
        Technology          54834526  251991.8320      2335     80.20   

                             Profit  Profitable  Expected Ship Time  \
Region  Category                                                      
Central Furniture        -2871.0494       157.0                2180   
        Office Supplies   8879.9799      1044.0                6579   
        Technology       33697.4320       370.0                1868   
East    Furniture         3046.1658       408.0                2631   
        Office Supplies  41014.5791      1469.0                7599   
        Technology       47462.0351       399.0                2378   
South   Furniture         6771.2061       267.0                1505   
        Office Supplies  19986.3928       821.0                4492   
        Technology       19991.8314       260.0                1302   
West    Furniture        11504.9503       542.0                3107   
        Office Supplies  52609.8490      1777.0                8434   
        Technology       44303.6496       544.0                2644   

                         Actual Ship Time   Late  
Region  Category                                  
Central Furniture                    1914  103.0  
        Office Supplies              5859  305.0  
        Technology                   1654   95.0  
East    Furniture                    2314  121.0  
        Office Supplies              6741  352.0  
        Technology                   2077   90.0  
South   Furniture                    1336   69.0  
        Office Supplies              3936  192.0  
        Technology                   1140   57.0  
West    Furniture                    2744  145.0  
        Office Supplies              7468  403.0  
        Technology                   2375  137.0

In [355]:
df.groupby(['Region','Category'])['Sales'].agg(['mean','min','max'])

mean    min        max
Region  Category                                     
Central Furniture        340.534644  1.892   3504.900
        Office Supplies  117.458801  0.444   9892.740
        Technology       405.753124  1.980  17499.950
East    Furniture        346.574383  2.960   4416.174
        Office Supplies  120.044425  0.852   4663.736
        Technology       495.278469  2.970  11199.968
South   Furniture        353.309289  2.784   4297.644
        Office Supplies  126.282727  1.167   6354.950
        Technology       507.753952  1.584  22638.480
West    Furniture        357.302325  3.480   3610.848
        Office Supplies  116.422377  1.080   8187.650
        Technology       420.687533  0.990  13999.960

In [356]:
#RESAMPLE
df.resample('w',on='Order Date')['Sales'].sum() #'w' = week; 'm'=month ; 'y'=year 
#you can even define what day you want the week the start on

Order Date
2014-01-05      324.0440
2014-01-12     4599.5720
2014-01-19     4509.1270
2014-01-26     3842.3880
2014-02-02     1642.3100
                 ...    
2017-12-03    32354.5700
2017-12-10    24006.9580
2017-12-17    10495.9630
2017-12-24    23662.3640
2017-12-31     8977.8318
Freq: W-SUN, Name: Sales, Length: 209, dtype: float64

**Question 1:** What salesperson had the highest average sales amount? 

In [423]:
df.groupby('Person')['Sales'].mean()

Person
Anna Andreadi        226.493233
Cassandra Brandow    241.803645
Chuck Magee          238.336110
Kelly Williams       215.772661
Name: Sales, dtype: float64

In [425]:
#for it to return the name, you would want it to return the index value so add ".idmax()"
df.groupby('Person')['Sales'].mean().idxmax()

'Cassandra Brandow'

**Question 2:** Within each ship mode, compare how likely late orders were to be profitable or not

In [371]:
df.groupby(['Ship Mode','Late'])['Profitable'].sum()

Ship Mode       Late 
First Class     False     727.0
                True      508.0
Same Day        False     422.0
                True       22.0
Second Class    False     968.0
                True      662.0
Standard Class  False    4249.0
                True      500.0
Name: Profitable, dtype: float64

**Question 3:** What ship mode had the most consistently on time orders?

In [430]:
df.groupby('Ship Mode')['Late'].mean()

Ship Mode
First Class       0.405722
Same Day          0.044199
Second Class      0.411311
Standard Class    0.104055
Name: Late, dtype: float64

**Question 4:** For each sales person, get their average, median, max, and count of their sales.

In [383]:
df.groupby('Person')['Sales'].agg(['mean','median','max', 'count'])

,mean,median,max,count
Person,,,,
Anna Andreadi,226.493233,60.840,13999.960,3203
Cassandra Brandow,241.803645,54.594,22638.480,1620
Chuck Magee,238.336110,54.900,11199.968,2848
Kelly Williams,215.772661,45.980,17499.950,2323


**Question 5:** Group your dataset according to  `Region`, and `Category`, and then call the `describe()` method to get the summary statistics for each subgroup.

In [382]:
df.groupby(['Region','Category']).describe()

Postal Code                                       \
                              count          mean           std      min   
Region  Category                                                           
Central Furniture             481.0  65486.453222  11200.602622  46060.0   
        Office Supplies      1422.0  65275.734880  11481.099454  46060.0   
        Technology            420.0  65393.026190  11530.132828  46060.0   
East    Furniture             601.0  17317.247920  12542.460289   1040.0   
        Office Supplies      1712.0  17401.548481  12657.699221   1453.0   
        Technology            535.0  18640.338318  13202.282930   1841.0   
South   Furniture             332.0  34552.117470  10839.698410  22153.0   
        Office Supplies       995.0  34243.389950  10595.433133  22153.0   
        Technology            293.0  35261.351536  12440.300306  22153.0   
West    Furniture             707.0  91680.442716   5024.265567  59601.0   
        Office Supplies      1897.0  91769.716921   5180.811997  59102.0   
        Technology            599.0  91543.449082   5473.761207  59405.0   

                                                              Sales  \
                             25%      50%      75%      max   count   
Region  Category                                                      
Central Furniture        56301.0  62521.0  77041.0  79907.0   481.0   
        Office Supplies  55122.0  65807.0  77041.0  79907.0  1422.0   
        Technology       55083.5  65807.0  77041.0  79907.0   420.0   
East    Furniture        10011.0  11561.0  19143.0  45503.0   601.0   
        Office Supplies  10011.0  11535.0  19143.0  45503.0  1712.0   
        Technology       10011.0  13501.0  19711.0  45503.0   535.0   
South   Furniture        28314.0  33023.5  37918.0  72701.0   332.0   
        Office Supplies  28403.0  32712.0  37211.0  72762.0   995.0   
        Technology       28403.0  32216.0  37211.0  72701.0   293.0   
West    Furniture        90032.0  92105.0  94521.0  99301.0   707.0   
        Office Supplies  90032.0  92374.0  94513.0  99301.0  1897.0   
        Technology       90008.0  92037.0  94513.0  99207.0   599.0   

                                     ... Expected Ship Time       \
                               mean  ...                75%  max   
Region  Category                     ...                           
Central Furniture        340.534644  ...                6.0  6.0   
        Office Supplies  117.458801  ...                6.0  6.0   
        Technology       405.753124  ...                6.0  6.0   
East    Furniture        346.574383  ...                6.0  6.0   
        Office Supplies  120.044425  ...                6.0  6.0   
        Technology       495.278469  ...                6.0  6.0   
South   Furniture        353.309289  ...                6.0  6.0   
        Office Supplies  126.282727  ...                6.0  6.0   
        Technology       507.753952  ...                6.0  6.0   
West    Furniture        357.302325  ...                6.0  6.0   
        Office Supplies  116.422377  ...                6.0  6.0   
        Technology       420.687533  ...                6.0  6.0   

                        Actual Ship Time                                     \
                                   count      mean       std  min  25%  50%   
Region  Category                                                              
Central Furniture                  481.0  3.979210  1.713787  0.0  3.0  4.0   
        Office Supplies           1422.0  4.120253  1.707383  0.0  3.0  4.0   
        Technology                 420.0  3.938095  1.586331  0.0  3.0  4.0   
East    Furniture                  601.0  3.850250  1.747628  0.0  3.0  4.0   
        Office Supplies           1712.0  3.937500  1.724834  0.0  3.0  4.0   
        Technology                 535.0  3.882243  1.720977  0.0  3.0  4.0   
South   Furniture                  332.0  4.024096  1.690392  0.0  3.0  4.0   
      

**Question 6:** Use the `Resample()` method to get the sum of sales for each quarter.

In [389]:
df.resample('q',on='Order Date')['Sales'].sum()

Order Date
2014-03-31     74447.7960
2014-06-30     86538.7596
2014-09-30    143633.2123
2014-12-31    179627.7302
2015-03-31     68851.7386
2015-06-30     89124.1870
2015-09-30    130259.5752
2015-12-31    182297.0082
2016-03-31     93237.1810
2016-06-30    136082.3010
2016-09-30    143787.3622
2016-12-31    236098.7538
2017-03-31    123144.8602
2017-06-30    133764.3720
2017-09-30    196251.9560
2017-12-31    280054.0670
Freq: Q-DEC, Name: Sales, dtype: float64

**Question 7:** What quarter had the highest total sales amount?

In [432]:
df.resample('q',on='Order Date')['Sales'].sum().max()

280054.06700000004

In [436]:
#would return the Index (quarter)
df.resample('q',on='Order Date')['Sales'].sum().idxmax()

Timestamp('2017-12-31 00:00:00', freq='Q-DEC')

In [438]:
#to have it on the same line 
grouping = df.resample('q',on='Order Date')['Sales'].sum().reset_index()
max_idx = grouping['Sales'].idxmax()
grouping.iloc[max_idx]

Order Date    2017-12-31 00:00:00
Sales                      280054
Name: 15, dtype: object

**Question 8:** See if you can use the `groupby` method to get a list of yearly sales for each region inside the dataset.

**Hint:** Try using the `dt` attribute of the `Order Date` column.

In [443]:
#need to put order date into a data frame so you can apply dt to it 
#dt only works on series 
df.groupby(['Region', df['Order Date'].dt.year])['Sales'].sum()


Region   Order Date
Central  2014          103838.1646
         2015          102874.2220
         2016          147429.3760
         2017          147098.1282
East     2014          128680.4570
         2015          156332.0570
         2016          180685.8220
         2017          213082.9040
South    2014          103845.8435
         2015           71359.9805
         2016           93610.2235
         2017          122905.8575
West     2014          147883.0330
         2015          139966.2495
         2016          187480.1765
         2017          250128.3655
Name: Sales, dtype: float64

**Bonus:** Creating summary statistics with a `groupby` statement.  

Lots of times it's very useful to be able to create a summary statistic for a particular category to compare with individual samples.

For example, if you were doing fraud detection, and someone were making a purchase at a 7-11, a transaction amount of $175 would be unusually large for someone making a purchase at a convenience store, essentially setting off a red flag that the transaction might be suspicious.

Creating such comparisons is easily done using the `groupby` method and then merging it back into the original dataframe.  

For example, if you wanted to compare every single purchase amount with the average amount for that category, you could do it in the following way:

In [447]:
# create the grouping
cat_grouping = df.groupby('Category')[['Sales']].mean()
# this step is mostly just to make the merged dataframe more tidy
cat_grouping.rename({'Sales': 'Cat_Average'}, axis=1, inplace=True)

In [448]:
# join them
df = df.merge(cat_grouping, left_on='Category', right_index=True)

In [449]:
# and now we can see each purchase amount compared to the average amt
# for that category
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,...,Quantity,Discount,Profit,Profitable,Expected Ship Time,Actual Ship Time,Late,Returned,Person,Cat_Average
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,2,0.00,41.9136,True,3,3,False,No,Cassandra Brandow,349.834887
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,3,0.00,219.5820,True,3,3,False,No,Cassandra Brandow,349.834887
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,5,0.45,-383.0310,False,6,7,True,No,Cassandra Brandow,349.834887
5,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,7,0.00,14.1694,True,6,5,False,No,Anna Andreadi,349.834887
10,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,...,9,0.20,85.3092,True,6,5,False,No,Anna Andreadi,349.834887


So for instance, if wanted to ask ourselves, "which customers consistently punch above their weight when it comes to the actual items that they buy?"

We could easily do the following:

In [450]:
# turn the difference between the two columns into a percent
df['Cat Difference'] = ((df['Sales'] / df['Cat_Average']) - 1) * 100

In [451]:
# now group and sort the values
df.groupby('Customer Name')['Cat Difference'].mean().sort_values(ascending=False)

Customer Name
Mitch Willingham        845.569285
Christopher Martinez    636.313301
Andy Reiter             449.654450
Adrian Barton           418.548520
Sanjit Chand            386.906876
                           ...    
Susan Gilcrest          -91.963736
Roy Skaria              -93.762646
Lela Donovan            -95.554963
Mitch Gastineau         -96.387490
Thais Sissman           -97.974843
Name: Cat Difference, Length: 793, dtype: float64

**Your Turn:** Using a similar methodology as above, figure out the 10 customers who are the most profitable on average, when compared to the subcategory that they purchased from.

If you wanted, you could also limit this to customers who only made a minimum number of purchases as well.

In [456]:
sub_catgroup = df.groupby('Sub-Category')[['Sales']].mean().rename({'Sales':'Subcat Avg'}, axis=1)

In [457]:
sub_catgroup

,Subcat Avg
Sub-Category,
Accessories,215.974604
Appliances,230.755710
Art,34.068834
Binders,133.560560
Bookcases,503.859633
Chairs,532.332420
Copiers,2198.941618
Envelopes,64.867724
Fasteners,13.936774


In [461]:
#compares sales amount with the items that they're purchasing to understand value 
df=df.merge(sub_catgroup, left_on="Sub-Category", right_index=True) #merge on the right index

In [462]:
df[['Sales','Subcat Avg']]

,Sales,Subcat Avg
0,261.9600,503.859633
27,3083.4300,503.859633
38,532.3992,503.859633
189,899.1360,503.859633
192,626.3520,503.859633
...,...,...
8820,959.9840,2198.941618
8990,4899.9300,2198.941618
9617,899.9820,2198.941618
9839,479.9760,2198.941618


In [464]:
#turn into percent to find the percent diff

df['Subcat Percent'] = (df['Sales']/df['Subcat Avg'])*100

In [469]:
df.groupby('Customer Name')['Subcat Percent'].mean().sort_values(ascending=False)

#Most price sensitive customers are closer to 0

Customer Name
Mitch Willingham        772.210825
Christopher Martinez    655.127205
Jenna Caffey            635.085743
Andy Reiter             496.344151
Adrian Barton           496.321356
                           ...    
Lela Donovan             15.568481
Susan Gilcrest           15.026927
Cari Schnelling          14.711453
Mitch Gastineau           8.082646
Thais Sissman             1.809292
Name: Subcat Percent, Length: 793, dtype: float64